<a href="https://colab.research.google.com/github/RAJESHVELAMALA/Project3/blob/main/ImageProcessingModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from PIL import Image
import pandas as pd
import requests
import numpy as np
import pickle
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from google.colab import drive
drive.mount('/content/drive')
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import load_model



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [59]:
def loadImages(path, filename):
  print("Loading images...")
  print("Reading the meta file containing all image file names")
  filenames_df = pd.read_csv(path+filename)
  base_path = path + "Images/"
  print(filenames_df.shape)
 # Create a empty list to hold images
  images = []
  filenames_df_success = []

  # Sort Dataset on id
  #filenames_df = filenames_df.sort_values(by=['id'])
  number_of_images = len(filenames_df)
  #number_of_images = 25

  for i in range(number_of_images):
      filename = filenames_df.iloc[i,0]
      path = base_path + filename+".jpg"
      print(f'{i} of {len(filenames_df)}: Attempting to import {path}')
      try:
          images.append(Image.open(path))
          filenames_df_success.append(filename)
      except:
          print(f'FAILED: {filename}')
  print(len(filenames_df_success))
  print("Images loaded!" + str(len(images)))
  return (images,filenames_df_success)


#imageList,fileNameList = loadImages("/content/sample_data/", "FOODITEMS.csv")

In [42]:
def createPickle(images):
  print("Creating pickle file...")
  with open('/content/drive/My Drive/food_imgages_project3.pkl', 'wb') as file:
      pickle.dump(images, file)
      print("Pickle file created!")

In [43]:
def createPickleDF(filenames):
  print("Creating pickle file...")
  filenames_df = pd.DataFrame(filenames)
  pd.to_pickle(filenames_df, '/content/drive/My Drive/food_filenames_df_project3.pkl')
  print("Pickle file created!")


In [37]:
def loadImagesFromPickle():
  # Mount google drive
  from google.colab import drive
  drive.mount('/content/drive')
  print("Loading images from pickle file...")
  with open('/content/drive/My Drive/food_imgages_project3.pkl', 'rb') as file:
      images = pickle.load(file)
  return images

In [36]:
def loadFilenamesFromPickle():
  from google.colab import drive
  drive.mount('/content/drive')

  print("Loading filenames from pickle file...")
  filenames_df = pd.read_pickle('/content/drive/My Drive/food_filenames_df_project3.pkl')
  return filenames_df

In [46]:
def preProcessImages(imageList):
  print("Preprocessing images...")
  # Resize images
  target_size = (160, 160)
  resized_images = [img.resize(target_size, resample = Image.LANCZOS) for img in faceImages]

  # Convert all images to floating point numpy arrays
  float_images = [np.array(img).astype(np.float32) for img in resized_images]

  # To normalize images to a range between 0 and 1,
  # we need to divide all pixel values by the max of 255

  normalized_images = [img/255 for img in float_images]

  # Display the pixel values of the first image
  #print(normalized_images[0])

  return normalized_images

In [47]:
def enhanceDataSetForAugmentation(X_train,y_train):
  # Apply augmentation to the whole training dataset
  # Create an ImageDataGenerator
  datagen = ImageDataGenerator(
      rotation_range=20,      # Random rotation (degrees)
      width_shift_range=0.1,  # Random horizontal shift
      height_shift_range=0.1, # Random vertical shift
      shear_range=0.2,        # Shear intensity
      zoom_range=0.2,         # Random zoom
      horizontal_flip=True,   # Random horizontal flip
      vertical_flip=False,    # No vertical flip for face images
      fill_mode='nearest'     # Fill mode for handling newly created pixels
  )

  # Create variables to hold the X and y training data
  X_train_aug = []
  y_train_aug = []
  # Loop through all the images.
  for i in range(len(X_train)):
      # Select the image
      img = X_train[i]
      # Select the label from the training data
      label = y_train[i]

      # Add a channel dimension for grayscale images if needed
      if img.ndim == 2:  # Check if it's grayscale (2 dimensions)
          img = np.expand_dims(img, axis=-1)  # Add channel dimension

      # Ensure that the input data has the correct shape (remove extra dimension)
      # img = np.squeeze(img, axis=-1)  # Remove the extra dimension - Not needed anymore

      # Add 5 images for every original image
      for j in range(5):
          # Append a new image to the X list
          X_train_aug.append(next(datagen.flow(np.expand_dims(img, axis=0), batch_size=1))[0]) # Use next() to get the next item from the iterator
          # Append the label for the original image to the y list - replicate the label for each augmented image
          y_train_aug.append(label)

  return X_train_aug, y_train_aug

In [48]:
def createModel(X_train_aug, y_train_aug,epochs):
  import tensorflow as tf
  from tensorflow import keras
  from tensorflow.keras import layers
  from tensorflow.keras.preprocessing.image import ImageDataGenerator
  from tensorflow.keras.optimizers import Adam
  from sklearn.model_selection import train_test_split
  from sklearn.preprocessing import OneHotEncoder
  import numpy as np

  # One hot encode the y data
  y_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False).fit(np.array(y_train_aug).reshape(-1, 1))
  y_train_aug_enc = y_encoder.transform(np.array(y_train_aug).reshape(-1, 1))


  # Convert values to numpy arrays
  X_train_aug_np = np.array(X_train_aug)
  y_train_aug_np = np.array(y_train_aug_enc)

  # Load and preprocess your CMU Face Images dataset (Ensure each image is labeled as "with sunglasses" or "without sunglasses")
  # The following code assumes that you have already loaded and preprocessed your dataset into 'X' and 'y' (features and labels).

  # Split the training dataset into training and validation sets
  X_train, X_val, y_train, y_val = train_test_split(X_train_aug_np, y_train_aug_np, test_size=0.2, random_state=42)

  # Define a CNN model
  #model = keras.Sequential([
  #    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(160, 160, 3)),
  ##    layers.MaxPooling2D((2, 2)),
  #    layers.Conv2D(64, (3, 3), activation='relu'),
  #    layers.MaxPooling2D((2, 2)),
  #    layers.Conv2D(64, (3, 3), activation='relu'),
  #    layers.Flatten(),
  #    layers.Dense(64, activation='relu'),
  ##    layers.Dense(64, activation='relu'),
  #    layers.Dense(18, activation='sigmoid')  # Change to 18 classes to match your labels
  #])

  print(type(y_train))
  print(y_train.shape[0],y_train.shape[1])
  num_classes = y_train.shape[1]


  # Define a CNN model
  model = keras.Sequential([
      layers.Conv2D(32, (3, 3), activation='relu', input_shape=(160, 160, 3)),
      #layers.MaxPooling2D((2, 2)),
      layers.Conv2D(64, (3, 3), activation='relu'),
      #layers.MaxPooling2D((2, 2)),
      #layers.Conv2D(64, (3, 3), activation='relu'),
      layers.Flatten(),
      layers.Dense(64, activation='relu',kernel_regularizer=l2(0.0001)),
      layers.Dense(num_classes, activation='sigmoid')  # Change to a single output node with sigmoid activation
  ])
  model.add(Dropout(0.5))  # Add dropout after a dense layer


  reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,patience=5, min_lr=0.001)
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  #model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, callbacks=[reduce_lr])

  # Compile the model
  #model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

  # Train the model
  batch_size = 10
  epochs = epochs
  history = model.fit(
      X_train, y_train,
      validation_data=(X_val, y_val),
      epochs=epochs,
      callbacks=[reduce_lr],
      batch_size=batch_size
  )

  return model,y_encoder


In [61]:
#imageList,fileNameList = loadImages("/content/sample_data/", "Dataset.csv")
imageList,fileNameList = loadImages("/content/sample_data/", "FOODITEMS.csv")

createPickle(imageList)
createPickleDF(fileNameList)


Loading images...
Reading the meta file containing all image file names
(100, 1)
0 of 100: Attempting to import /content/sample_data/Images/baby_back_ribs.jpg
1 of 100: Attempting to import /content/sample_data/Images/baklava.jpg
2 of 100: Attempting to import /content/sample_data/Images/beef_carpaccio.jpg
3 of 100: Attempting to import /content/sample_data/Images/beef_tartare.jpg
4 of 100: Attempting to import /content/sample_data/Images/beet_salad.jpg
5 of 100: Attempting to import /content/sample_data/Images/beignets.jpg
6 of 100: Attempting to import /content/sample_data/Images/bibimbap.jpg
7 of 100: Attempting to import /content/sample_data/Images/bread_pudding.jpg
8 of 100: Attempting to import /content/sample_data/Images/breakfast_burrito.jpg
9 of 100: Attempting to import /content/sample_data/Images/bruschetta.jpg
10 of 100: Attempting to import /content/sample_data/Images/caesar_salad.jpg
11 of 100: Attempting to import /content/sample_data/Images/cannoli.jpg
12 of 100: Attemp

In [67]:
# Now we'll split our data into training and testing sets


#Load Images from pickle file
faceImages = loadImagesFromPickle()
filenames = loadFilenamesFromPickle()
filenames_df = pd.DataFrame(filenames)
#sizes = set([img.size for img in faceImages])

# Now we can call our preprocessed pixel data 'X'
X = preProcessImages(faceImages)

filenames_df['username'] = filenames_df[0]

#filenames_df[['username', 'number']] = filenames_df[0]\
#                                                            .str.replace('.jpg', '', regex=False)\
#                                                            .str.split('_', expand=True)
print(filenames_df.head())

y = np.array(filenames_df['username'])
print("Number of classes", len(filenames_df['username'].unique()))

X_train, X_test, y_train, y_test = train_test_split(X, y)


X_train_aug, y_train_aug = enhanceDataSetForAugmentation(X_train,y_train)
model,y_encoder = createModel(X_train_aug, y_train_aug,20)
#model,y_encoder = createModel(X_train, y_train,20)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loading images from pickle file...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loading filenames from pickle file...
Preprocessing images...
                0        username
0  baby_back_ribs  baby_back_ribs
1         baklava         baklava
2  beef_carpaccio  beef_carpaccio
3    beef_tartare    beef_tartare
4      beet_salad      beet_salad
Number of classes 100
<class 'numpy.ndarray'>
300 75


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 144s 5s/step - accuracy: 0.0030 - loss: 11.4936 - val_accuracy: 0.0133 - val_loss: 4.4580 - learning_rate: 0.0010
Epoch 2/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 133s 4s/step - accuracy: 0.0394 - loss: 9.5894 - val_accuracy: 0.0133 - val_loss: 4.4612 - learning_rate: 0.0010
Epoch 3/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 144s 4s/step - accuracy: 0.0739 - loss: 10.5319 - val_accuracy: 0.0000e+00 - val_loss: 4.6718 - learning_rate: 0.0010
Epoch 4/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 141s 5s/step - accuracy: 0.1178 - loss: 9.4374 - val_accuracy: 0.0133 - val_loss: 4.3834 - learning_rate: 0.0010
Epoch 5/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 141s 5s/step - accuracy: 0.1381 - loss: 10.0158 - val_accuracy: 0.0533 - val_loss: 4.8925 - learning_rate: 0.0010
Epoch 6/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 136s 4s/step - accuracy: 0.1876 - loss: 9.7422 - val_accuracy: 0.0933 - val_loss: 4.3652 - learning_rate: 0.0010
Epoch 7/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 142s 4s/step - accuracy: 0.2915 - loss: 10.7780 -

In [68]:
# Reshape test data for the model
X_test_np = []
for img in X_test:
    # Ensure the image has 3 channels (assuming color images)
    if img.ndim == 2:  # If grayscale, convert to 3 channels
        img = np.stack((img,) * 3, axis=-1)
    # Append the image to the list
    X_test_np.append(img)

# Convert to numpy array
X_test_np = np.array(X_test_np)

# One hot encode the y data
y_test_enc = y_encoder.transform(np.array(y_test).reshape(-1, 1))
y_test_np = np.array(y_test_enc)

print(y_test_np[0].shape, '#', y_test_np.shape)
print(X_test_np[0].shape, '#', X_test_np.shape)

# Once the number of samples match, you can evaluate the model:
if X_test_np.shape[0] == y_test_np.shape[0]:
  model.evaluate(X_test_np, y_test_np)
else:
  print("Error: Number of samples in X_test_np and y_test_np don't match. Cannot evaluate the model.")

(75,) # (25, 75)
(160, 160, 3) # (25, 160, 160, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0000e+00 - loss: 0.3357


In [49]:
# Save the entire model
model.save('/content/drive/My Drive/food_model.h5')

In [50]:
# Load the model
loaded_model = load_model('/content/drive/My Drive/food_model.h5')

# Check the model summary to confirm it loaded correctly
loaded_model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_14 (Conv2D)                   │ (None, 158, 158, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_15 (Conv2D)                   │ (None, 156, 156, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_7 (Flatten)                  │ (None, 1557504)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 64)                  │      99,680,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 75)                  │           4,875 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 75)                  │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 99,704,589 (380.34 MB)

 Trainable params: 99,704,587 (380.34 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [54]:
from PIL import Image
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

# Load and preprocess the image
#image_path = '/content/drive/MyDrive/sampleimages/sushi.jpg'
image_path = '/content/drive/MyDrive/sampleimages/tacos.jpg'
#image_path = '/content/drive/MyDrive/sampleimages/takoyaki.jpg'
image = Image.open(image_path)
image = image.resize((160, 160))
image_array = img_to_array(image)
image_array = np.expand_dims(image_array, axis=0)
image_array = image_array / 255.0

# Load the trained model
model = load_model('/content/drive/My Drive/food_model.h5')
class_labels = filenames_df['foodName'].tolist()

# Make a prediction
predictions = model.predict(image_array)
predicted_class = np.argmax(predictions, axis=1)
print(predicted_class)

#print(class_labels)
predicted_label = class_labels[predicted_class[0]]

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 848ms/step
[65]


In [55]:

filenames = loadFilenamesFromPickle()
filenames_df = pd.DataFrame(filenames)


filenames_df['foodName']=filenames_df[0:]

# Example class labels
class_labels = filenames_df['foodName'].tolist()

#print(class_labels)
predicted_label = class_labels[predicted_class[0]]

print(f'Predicted label: {predicted_label}')

#

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loading filenames from pickle file...
Predicted label: nachos


In [46]:
filenames_df.loc[filenames_df['foodName'] == 'sushi']

,0,foodName
94,sushi,sushi
